# Chapter 5.2 - Matrix Multiplication in Spark using RDDs

In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [58]:
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

### Matrix Multiplication Reviewed
* Critical to a large number of tasks from graphics and cryptography to graph algorithms and machine learning.
* Computationally intensive. A naive sequential matrix multiplication algorithm has complexity of O(n^3). 
* Algorithms with lower computational complexity exist, but they are not always faster in practice.
* Good candidate for distributed processing

* Every matrix cell is computed using a separate, independent from other cells computation. The computation consumes O(n) input (one matrix row and one matrix column).
* Good candidate for being expressed as a MapReduce computation.
* For a refresher on matrix muliplication. <a href="https://www.youtube.com/watch?v=kuixY2bCc_0&ab_channel=MathMeeting">Here is one such video.</a>
* <a href="https://en.wikipedia.org/wiki/Matrix_multiplication#Definition">Formal definition</a>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/18/Matrix_multiplication_qtl1.svg/2560px-Matrix_multiplication_qtl1.svg.png">

### Why Spark for matrix multiplication? 
If you've ever tried to perform matrix multiplication and you've run out of memory, then you know one of the reasons we might want to use Spark. In general, it is faster to work with a library such as numpy when the matrices are reasonable in size. We would only see the performance benefits of a Spark approach at scale.

### Creating our input

Creating the input for testing purposes is easy. In practice, we would be reading from files or a database. Please review the documentation on <a href="https://spark.apache.org/docs/2.1.1/programming-guide.html#parallelized-collections">parallelized collections</a>.

Let $A$ be a matrix of size $m \times n$ and $B$ be a matrix of size $n \times s$. Then our goal is to create a matrix $R$ of size $m \times s$. 

Let's start with a concrete example that is represented in what seems like a reasonable way. In general, we use two dimensional arrays to represent lists. Things like:
```python
[[1,2,3],[4,5,6]]
```
We will do that here, but we will write each row as a key,value pair such as:
```python
[('A[1,:]',[1,2,3]),
 ('A[2,:]',[4,5,6])]
```
We'll switch to different formats later for reasons that you will notice while doing this first exercise. If you haven't seen ``A[1,:]`` it means this is the first row and all the columns of the A matrix. Below is how we create the RDDs.

In [59]:
A = [('A[1,:]',[1, 2, 3]),('A[2,:]',[4, 5,6])]
A_RDD = sc.parallelize(A)

B = [('B[:,1]',[7,9,11]),('B[:,2]',[8,10,12])]
B_RDD = sc.parallelize(B)

We can convert these into numpy arrays easily.

In [60]:
import numpy as np
A_mat = np.array(A_RDD.map(lambda v: v[1]).collect())
A_mat

array([[1, 2, 3],
       [4, 5, 6]])

In [61]:
B_mat = np.array(B_RDD.map(lambda v: v[1]).collect())
B_mat

array([[ 7,  9, 11],
       [ 8, 10, 12]])

Let's ask numpy to do our multiplication for us. **Error below is on purpose**. The dot product between two vectors:
<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/5bd0b488ad92250b4e7c2f8ac92f700f8aefddd5">
So numpy will calculate the dot product of two vectors each time an entry (circle in image below) is needed:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Matrix_multiplication_diagram_2.svg/440px-Matrix_multiplication_diagram_2.svg.png">

In [62]:
np.dot(A_mat,B_mat)

ValueError: shapes (2,3) and (2,3) not aligned: 3 (dim 1) != 2 (dim 0)

We have already transposed B in our example to make our map reduce task easier. The ``.dot`` function assumes we have not done the transpose. So in order for numpy to do the multiplication for us, we need to transpose the second matrix (note the .T).

In [8]:
np.dot(A_mat,B_mat.T)

array([[ 58,  64],
       [139, 154]])

Let's pick apart how we got the value 64. This is the dot product of row 1 of A and column 2 of B (Hint: when we discuss matrix we start counting at 1). Our goal then is to compute $n \times s$ values: one for each pair (i, k) of rows from matrix A and columns from matrix B.

To do this we'll need to join the two RDDs together. 

**Stop and think:** Why is the following empty?

In [11]:
A_RDD.join(B_RDD).collect()

[]

**Your solution here**

Here is the output of the cartesian product:

In [12]:
A_RDD.cartesian(B_RDD).collect()

[(('A[1,:]', [1, 2, 3]), ('B[:,1]', [7, 9, 11])),
 (('A[1,:]', [1, 2, 3]), ('B[:,2]', [8, 10, 12])),
 (('A[2,:]', [4, 5, 6]), ('B[:,1]', [7, 9, 11])),
 (('A[2,:]', [4, 5, 6]), ('B[:,2]', [8, 10, 12]))]

**Stop and think:** Can you calculate entry R[2,1]? Take a moment and do so on paper.

For reference, here is the numpy calculated answers:

In [16]:
R = np.dot(A_mat,B_mat.T)
R[2-1,1-1] # -1 is to transform mathematical notation to 0-based indexing

139

$$
(4, 5, 6) \cdot (7, 9, 11) = 4*7+5*9+6*11 = 139
$$

We will now get Spark to calculate the matrix multiplication for us.

In [18]:
# These functions are needed because we used a string above 
# In the second implementation of matrix muliplication, we don't need to do this
def get_row(s):
    return int(s.split(",")[0].split("[")[-1])

def get_col(s):
    return int(s.split(",")[1].split("]")[0])

In [19]:
A2 = A_RDD.map(lambda kv: (get_row(kv[0]),kv[1]))
A2.collect()

[(1, [1, 2, 3]), (2, [4, 5, 6])]

In [20]:
B2 = B_RDD.map(lambda kv: (get_col(kv[0]),kv[1]))
B2.collect()

[(1, [7, 9, 11]), (2, [8, 10, 12])]

**Exercise 1:**
Using what I have defined above (A_RDD, B_RDD, A2, B2), the Spark functions (cartesian, map, collect), and the numpy function (np.dot or a loop of your own), compute the matrix multiplication of A_RDD and B_RDD. Here is a reminder of what cartesian results in:

In [22]:
A2.cartesian(B2).collect()

[((1, [1, 2, 3]), (1, [7, 9, 11])),
 ((1, [1, 2, 3]), (2, [8, 10, 12])),
 ((2, [4, 5, 6]), (1, [7, 9, 11])),
 ((2, [4, 5, 6]), (2, [8, 10, 12]))]

In [23]:
def exercise_1(A_RDD,B_RDD):
    result = None
    A2 = A_RDD.map(lambda kv: (get_row(kv[0]),kv[1]))
    B2 = B_RDD.map(lambda kv: (get_col(kv[0]),kv[1]))

    # Your solution here
    return result

result = exercise_1(A_RDD,B_RDD)
result

[((1, 1), 58), ((1, 2), 64), ((2, 1), 139), ((2, 2), 154)]

**We did it!** Matrix multiplication using map reduce.

If you want to put it back in the same format, the keys provide that information:

In [24]:
R_mat = np.zeros((2,2))
for row_col,value in result:
    row,col = row_col
    R_mat[row-1,col-1] = value
R_mat

array([[ 58.,  64.],
       [139., 154.]])

**Exercise 2:** Implement matrix multiplication using the following alternative format:

'row number', 'column number', 'value'

For this exercise, you cannot use loops or np.dot. It should be Spark centric using cartesian, join, map, add, reduceByKey, and/or collect. 

In [34]:
A = [[1,1,1],
     [1,2,0],
     [2,1,3],
     [2,2,4],
     [3,1,0],
     [3,2,6],
     [4,1,7],
     [4,2,8]
    ]
A_RDD = sc.parallelize(A)

B = [[1,1,7],
     [1,2,8],
     [1,3,9],
     [2,1,0],
     [2,2,11],
     [2,3,0]
    ]
B_RDD = sc.parallelize(B)

We will now convert this to numpy arrays and perform the multiplication:

In [35]:
A_mat = np.zeros((4,2))
for row,col,val in A:
    A_mat[row-1,col-1]=val
A_mat

B_mat = np.zeros((2,3))
for row,col,val in B:
    B_mat[row-1,col-1]=val
A_mat,B_mat

(array([[1., 0.],
        [3., 4.],
        [0., 6.],
        [7., 8.]]),
 array([[ 7.,  8.,  9.],
        [ 0., 11.,  0.]]))

In [36]:
R = np.dot(A_mat,B_mat)
R

array([[  7.,   8.,   9.],
       [ 21.,  68.,  27.],
       [  0.,  66.,   0.],
       [ 49., 144.,  63.]])

Before coding, consider the following diagram and ask yourself the following when considering how to calculate the yellow circle:

**How would you join/match the correct elements (rows and columns) of A and B together to compute, R[1,2]?**

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Matrix_multiplication_diagram_2.svg/440px-Matrix_multiplication_diagram_2.svg.png">

**Your answer here**

**Stop and think:** Can you write Spark code that joins the correct elements from A_RDD and B_RDD?

In [48]:
AB=None
# Your solution here
AB

[(1, ([1, 1, 1], [1, 1, 7])),
 (1, ([1, 1, 1], [1, 2, 8])),
 (1, ([1, 1, 1], [1, 3, 9])),
 (1, ([2, 1, 3], [1, 1, 7])),
 (1, ([2, 1, 3], [1, 2, 8])),
 (1, ([2, 1, 3], [1, 3, 9])),
 (1, ([3, 1, 0], [1, 1, 7])),
 (1, ([3, 1, 0], [1, 2, 8])),
 (1, ([3, 1, 0], [1, 3, 9])),
 (1, ([4, 1, 7], [1, 1, 7])),
 (1, ([4, 1, 7], [1, 2, 8])),
 (1, ([4, 1, 7], [1, 3, 9])),
 (2, ([1, 2, 0], [2, 1, 0])),
 (2, ([1, 2, 0], [2, 2, 11])),
 (2, ([1, 2, 0], [2, 3, 0])),
 (2, ([2, 2, 4], [2, 1, 0])),
 (2, ([2, 2, 4], [2, 2, 11])),
 (2, ([2, 2, 4], [2, 3, 0])),
 (2, ([3, 2, 6], [2, 1, 0])),
 (2, ([3, 2, 6], [2, 2, 11])),
 (2, ([3, 2, 6], [2, 3, 0])),
 (2, ([4, 2, 8], [2, 1, 0])),
 (2, ([4, 2, 8], [2, 2, 11])),
 (2, ([4, 2, 8], [2, 3, 0]))]

**Now what?** Can we just reduce by key perform multiplication and then do a summation?

... This is a leading question. If you try this, we'll be heading down an incorrect path. Instead, is there something we can map the keys to a key that helps us out? My real question to you is:
**Pick a line at random, and what should be the key?**
<pre>
(1, ([3, 1, 0], [1, 2, 8]))
</pre>

**Your solution here**

Once you do it manually, then do it using Spark...

In [49]:
AB=None
AB

[((1, 1), [1, 7]),
 ((1, 2), [1, 8]),
 ((1, 3), [1, 9]),
 ((2, 1), [3, 7]),
 ((2, 2), [3, 8]),
 ((2, 3), [3, 9]),
 ((3, 1), [0, 7]),
 ((3, 2), [0, 8]),
 ((3, 3), [0, 9]),
 ((4, 1), [7, 7]),
 ((4, 2), [7, 8]),
 ((4, 3), [7, 9]),
 ((1, 1), [0, 0]),
 ((1, 2), [0, 11]),
 ((1, 3), [0, 0]),
 ((2, 1), [4, 0]),
 ((2, 2), [4, 11]),
 ((2, 3), [4, 0]),
 ((3, 1), [6, 0]),
 ((3, 2), [6, 11]),
 ((3, 3), [6, 0]),
 ((4, 1), [8, 0]),
 ((4, 2), [8, 11]),
 ((4, 3), [8, 0])]

**We can now put it all together and solve the problem!**

In [51]:
def exercise_2(A_RDD,B_RDD):
    result = None
    # Your solution here
    return result

result = exercise_2(A_RDD,B_RDD)
result

[((4, 2), 144),
 ((3, 1), 0),
 ((3, 3), 0),
 ((2, 3), 27),
 ((2, 2), 68),
 ((4, 3), 63),
 ((1, 2), 8),
 ((1, 3), 9),
 ((3, 2), 66),
 ((1, 1), 7),
 ((4, 1), 49),
 ((2, 1), 21)]

In [52]:
result_mat = np.zeros((4,3))
for row_col,val in result:
    row,col = row_col
    result_mat[row-1,col-1] = val
result_mat

array([[  7.,   8.,   9.],
       [ 21.,  68.,  27.],
       [  0.,  66.,   0.],
       [ 49., 144.,  63.]])

In [53]:
R == result_mat

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

This second format is very useful and common when representing sparse matrices. In this format, we remove/exclude any entry where a value in the matrix is 0. This occurs twice in A and twice in B. This results in a significant memory savings when a lot of the entries are in fact 0.

**Exercise 3:** Implement matrix multiplication using the following alternative format that assumes missing rows have a value of 0 (i.e., sparse matrices):

'row number', 'column number', 'value'

For this exercise, you cannot use loops or np.dot. It should be Spark centric using join, map, add, reduceByKey, and/or collect. 

In [54]:
A = [[1,1,1],
#     [1,2,0],
     [2,1,3],
     [2,2,4],
#     [3,1,0],
     [3,2,6],
     [4,1,7],
     [4,2,8]
    ]
A_RDD = sc.parallelize(A)

B = [[1,1,7],
     [1,2,8],
     [1,3,9],
#     [2,1,0],
     [2,2,11],
#     [2,3,0]
    ]
B_RDD = sc.parallelize(B)

In [55]:
def exercise_3(A_RDD,B_RDD):
    result = None
    # Your solution here
    return result

result = exercise_3(A_RDD,B_RDD)
result

[((4, 2), 144),
 ((2, 3), 27),
 ((2, 2), 68),
 ((4, 3), 63),
 ((1, 2), 8),
 ((1, 3), 9),
 ((3, 2), 66),
 ((1, 1), 7),
 ((4, 1), 49),
 ((2, 1), 21)]

In [56]:
result_mat = np.zeros((4,3))
for row_col,val in result:
    row,col = row_col
    result_mat[row-1,col-1] = val
result_mat

array([[  7.,   8.,   9.],
       [ 21.,  68.,  27.],
       [  0.,  66.,   0.],
       [ 49., 144.,  63.]])

**We didn't have to change a thing!**